In [1]:
import onnxruntime as rt
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

In [8]:
sess = rt.InferenceSession(
    "../data/models/yolov7/best.onnx", providers=rt.get_available_providers())

InvalidGraph: [ONNXRuntimeError] : 10 : INVALID_GRAPH : Load model from ../data/models/yolov7/best.onnx failed:This is an invalid model. In Node, ("batched_nms", EfficientNMS_TRT, "", -1) : ("output": tensor(float),) -> ("num_dets": tensor(int32),"det_boxes": tensor(float),"det_scores": tensor(float),"det_classes": tensor(int32),) , Error Node (batched_nms) has input size 1 not in range [min=2, max=3].

In [3]:
sess.get_inputs()[0].name, sess.get_outputs()[0].name

('images', 'output')

In [2]:
# tf.saved_model.LoadOptions(experimental_io_device="/job:localhost")
model = tf.saved_model.load("../data/models/yolov7/model/")

In [3]:
model.signatures['serving_default'].outputs

[<tf.Tensor 'Identity:0' shape=(1, 25200, 6) dtype=float32>]

In [4]:
image = cv2.imread("../data/input/train_shelf_images/train1.jpg")
image_resized = cv2.resize(image, (640, 640), 3)
image_array = np.array(image_resized)

In [5]:
image_array = image_array.reshape((3, 640, 640))

In [ ]:
sess.run(

In [6]:
tensor = tf.convert_to_tensor(image_array, dtype=tf.float32)

In [7]:
tensor[tf.newaxis, ...]

<tf.Tensor: shape=(1, 3, 640, 640), dtype=float32, numpy=
array([[[[184., 163., 166., ..., 100., 113., 229.],
         [216., 218., 226., ..., 167., 149., 175.],
         [181.,  60.,  80., ..., 202., 223., 220.],
         ...,
         [ 43.,  76.,  78., ...,  99.,  69.,  73.],
         [103.,  77.,  83., ...,  34.,  43., 164.],
         [ 53.,  56., 122., ...,  35.,  64.,  30.]],

        [[ 49.,  83.,  73., ..., 101.,  95.,  99.],
         [121., 133., 139., ...,  35.,  43., 164.],
         [ 71.,  80., 135., ...,  44.,  74.,  36.],
         ...,
         [210.,  29., 117., ...,  21.,  53.,  63.],
         [109., 147., 161., ..., 195., 217., 203.],
         [208., 211., 205., ..., 208.,  27., 116.]],

        [[210.,  29., 117., ...,  17.,  43.,  56.],
         [147., 167., 172., ..., 194., 218., 204.],
         [209., 212., 205., ..., 209.,  26., 116.],
         ...,
         [198., 203., 194., ...,  58.,  95.,  59.],
         [ 66., 109.,  61., ..., 171., 183., 180.],
         [17

In [8]:
detections = model(images=tensor[tf.newaxis, ...])

In [9]:
detections

{'output': <tf.Tensor: shape=(1, 25200, 6), dtype=float32, numpy=
 array([[[-3.9832613e+00,  1.1849958e+01,  1.2787511e+00,  1.4831218e-02,
           8.9960208e-09,  9.9999839e-01],
         [ 6.7149887e+00,  1.1966196e+01,  4.7811880e+00,  1.1477406e-02,
           9.0610009e-01,  9.9490482e-01],
         [ 1.2000011e+01,  1.1997852e+01,  4.3972259e+00,  2.7670932e-04,
           9.2065036e-03,  9.9999964e-01],
         ...,
         [ 5.8964642e+02,  5.9200000e+02,  2.8432675e+02,  1.0383432e+00,
           3.2867004e-15,  1.4161912e-02],
         [ 6.2025775e+02,  5.9200012e+02,  7.2099640e+01,  3.7697846e-01,
           1.3404424e-14,  2.4972254e-02],
         [ 6.4489551e+02,  5.9203613e+02,  7.1999290e+01,  2.2729480e+00,
           9.8389404e-12,  3.6396950e-01]]], dtype=float32)>}

In [65]:
inputs = model.signatures['serving_default'].inputs[0]
outputs = model.signatures['serving_default'].outputs

In [66]:
results = sess.run(outputs, {inputs: np.expand_dims(image, axis=0)})

TypeError: Tensor is unhashable. Instead, use tensor.ref() as the key.